In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from enum import Enum
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv('../03_preprocessing/naive_processed_features.csv', index_col=0)

In [4]:
df_motif = pd.read_csv('../03_preprocessing/motif_processed_features.csv', index_col=0)

In [5]:
# df = df[(df['neutral_vs_SARS-CoV2_WT'] != 2) & (df['binding_vs_SARS-CoV2_WT'] != 2)]

In [6]:
df.describe()

,neutral_vs_SARS-CoV2_WT,binding_vs_SARS-CoV2_WT,CDRH3_len,CDRH3_amino_acid_percentage_A,CDRH3_amino_acid_percentage_C,CDRH3_amino_acid_percentage_D,CDRH3_amino_acid_percentage_E,CDRH3_amino_acid_percentage_F,CDRH3_amino_acid_percentage_G,CDRH3_amino_acid_percentage_H,...,VHorVHH_amino_acid_percentage_V,VHorVHH_amino_acid_percentage_W,VHorVHH_amino_acid_percentage_Y,S_RBD,S_NTD,S_S2,S_S1,N_Protein,Other_Spike,Unknown
count,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,...,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000
mean,1.095758,1.030423,15.822728,0.104839,0.009948,0.114407,0.023903,0.057473,0.111540,0.017098,...,0.081481,0.029617,0.060832,0.703127,0.067116,0.071064,0.010760,0.007122,0.145843,0.002090
std,0.748488,0.356217,3.935180,0.061707,0.030721,0.061182,0.038948,0.052016,0.071956,0.033609,...,0.022509,0.009736,0.020906,0.456898,0.250232,0.256941,0.103176,0.084093,0.352962,0.045672
min,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,13.000000,0.062500,0.000000,0.071429,0.000000,0.000000,0.062500,0.000000,...,0.073770,0.024590,0.049587,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,15.000000,0.090909,0.000000,0.111111,0.000000,0.058824,0.105263,0.000000,...,0.084034,0.031250,0.062016,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,1.000000,18.000000,0.142857,0.000000,0.153846,0.052632,0.083333,0.153846,0.000000,...,0.094017,0.033613,0.074380,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,2.000000,63.000000,0.500000,0.230769,0.500000,0.272727,0.363636,0.444444,0.272727,...,0.140625,0.067797,0.132812,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
df_motif.describe()

,neutral_vs_SARS-CoV2_WT,binding_vs_SARS-CoV2_WT,CDRH3_CTD_111,CDRH3_CTD_112,CDRH3_CTD_113,CDRH3_CTD_114,CDRH3_CTD_115,CDRH3_CTD_116,CDRH3_CTD_117,CDRH3_CTD_121,...,VHorVHH_CTD_765,VHorVHH_CTD_766,VHorVHH_CTD_767,VHorVHH_CTD_771,VHorVHH_CTD_772,VHorVHH_CTD_773,VHorVHH_CTD_774,VHorVHH_CTD_775,VHorVHH_CTD_776,VHorVHH_CTD_777
count,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,12918.000000,...,12918.000000,12918.000000,1.291800e+04,12918.000000,12918.000000,12918.000000,1.291800e+04,12918.000000,12918.0,12918.0
mean,1.095758,1.030423,0.018534,0.013117,0.016124,0.005498,0.002901,0.006089,0.000502,0.008254,...,0.000004,0.000006,7.899119e-07,0.000005,0.000001,0.000006,4.635411e-07,0.000001,0.0,0.0
std,0.748488,0.356217,0.049963,0.030910,0.033776,0.020973,0.015135,0.021486,0.005806,0.026299,...,0.000188,0.000217,8.977931e-05,0.000205,0.000104,0.000216,5.268487e-05,0.000088,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0
25%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0
50%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0
75%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0
max,2.000000,2.000000,0.800000,0.400000,0.300000,0.333333,0.200000,0.222222,0.125000,0.285714,...,0.008333,0.008475,1.020408e-02,0.008621,0.008333,0.008475,5.988024e-03,0.007874,0.0,0.0


In [ ]:

exclude = ['neutral_vs_SARS-CoV2_WT', 'binding_vs_SARS-CoV2_WT', 'name']
X = df_motif[df_motif['neutral_vs_SARS-CoV2_WT'] != 2].drop(columns=exclude)
y = df_motif[df_motif['neutral_vs_SARS-CoV2_WT'] != 2]['neutral_vs_SARS-CoV2_WT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_antibody = RandomForestClassifier(n_estimators=100, class_weight='balanced')
rf_antibody.fit(X_train, y_train)

y_pred = rf_antibody.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.41      0.50       575
           1       0.75      0.89      0.81      1150

    accuracy                           0.73      1725
   macro avg       0.70      0.65      0.66      1725
weighted avg       0.72      0.73      0.71      1725



In [11]:
exclude = ['neutral_vs_SARS-CoV2_WT', 'binding_vs_SARS-CoV2_WT', 'name']
X = df_motif[df_motif['binding_vs_SARS-CoV2_WT'] != 2].drop(columns=exclude)
y = df_motif[df_motif['binding_vs_SARS-CoV2_WT'] != 2]['binding_vs_SARS-CoV2_WT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rf_antibody = RandomForestClassifier(n_estimators=100, class_weight='balanced')
rf_antibody.fit(X_train, y_train)

y_pred = rf_antibody.predict(X_test)
# 4. Results
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.10      0.17       126
           1       0.95      1.00      0.98      2254

    accuracy                           0.95      2380
   macro avg       0.98      0.55      0.57      2380
weighted avg       0.95      0.95      0.93      2380

